# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to 2015200703@mail.buct.edu.cn and will expire on November 11, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\SSQ\AppData\Local\Temp\graphlab_server_1481343536.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.497086     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.533088     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [8]:
rows = len(all_features)
model_all.get('coefficients').print_rows(num_rows=rows)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [9]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [13]:
import numpy as np
np_logspace = np.logspace(1,7,num=13)
num_penality = len(np_logspace)
RSS = np.empty(num_penality)
for i in xrange(num_penality):
    model = graphlab.linear_regression.create(training,target='price',features=all_features,l1_penalty=np_logspace[i],l2_penalty=0,validation_set=None,verbose=False)
    prediction = model.predict(validation)
    errors = prediction - validation['price']
    RSS[i] = np.dot(errors,errors)
print RSS

[  6.25766285e+14   6.25766285e+14   6.25766286e+14   6.25766288e+14
   6.25766295e+14   6.25766317e+14   6.25766387e+14   6.25766607e+14
   6.25767303e+14   6.25769508e+14   6.25776518e+14   6.25799063e+14
   6.25883719e+14]


In [27]:
rss_min = RSS.min()
#print rss_min
for i in xrange(num_penality):
    #print RSS[i]
    #print np_logspace[i]
    if RSS[i]==rss_min:
        print np_logspace[i]

10.0


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [35]:
model_l1 = graphlab.linear_regression.create(training,target='price',features=all_features,l1_penalty=10,l2_penalty=0,validation_set=None,verbose=False)
model_l1.get('coefficients').print_rows(num_rows=rows)
model_l1.coefficients['value'].nnz()

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

18

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [33]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [34]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [43]:
np_logspace = np.logspace(8, 10, num=20)
num_penality = len(np_logspace)
RSS = np.empty(num_penality)
for i in xrange(num_penality):
    model = graphlab.linear_regression.create(training,target='price',features=all_features,l1_penalty=np_logspace[i],l2_penalty=0,validation_set=None,verbose=False)
    prediction = model.predict(validation)
    errors = prediction - validation['price']
    RSS[i] = np.dot(errors,errors)
    nnz = model['coefficients']['value'].nnz()
    print str(i) + ': '+ str(nnz)
print RSS

0: 18
1: 18
2: 18
3: 18
4: 17
5: 17
6: 17
7: 17
8: 17
9: 16
10: 15
11: 15
12: 13
13: 12
14: 10
15: 6
16: 5
17: 3
18: 1
19: 1
[  6.27492660e+14   6.28210517e+14   6.29176690e+14   6.30650083e+14
   6.32940229e+14   6.36268140e+14   6.41261198e+14   6.48983455e+14
   6.60962218e+14   6.77261521e+14   7.01046816e+14   7.37850623e+14
   7.96163110e+14   8.69018173e+14   9.66925692e+14   1.08186759e+15
   1.24492736e+15   1.38416149e+15   1.23079472e+15   1.22915716e+15]


In [45]:
print np_logspace

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [47]:
l1_penalty_min = np_logspace[14]
l1_penalty_max = np_logspace[15]
print l1_penalty_min
print l1_penalty_max

2976351441.63
3792690190.73


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [48]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [49]:
num_l1 = len(l1_penalty_values)
RSS = np.empty(num_l1)
for i in xrange(num_l1):
    model = graphlab.linear_regression.create(training,target='price',features=all_features,l1_penalty=l1_penalty_values[i],l2_penalty=0,validation_set=None,verbose=False)
    prediction = model.predict(validation)
    errors = prediction - validation['price']
    RSS[i] = np.dot(errors,errors)
    nnz = model['coefficients']['value'].nnz()
    print str(i) + ': '+ str(nnz)
print RSS   

0: 10
1: 10
2: 10
3: 10
4: 10
5: 10
6: 10
7: 10
8: 8
9: 8
10: 8
11: 7
12: 7
13: 7
14: 7
15: 6
16: 6
17: 6
18: 6
19: 6
[  9.66925692e+14   9.74019450e+14   9.81188368e+14   9.89328342e+14
   9.98783211e+14   1.00847717e+15   1.01829878e+15   1.02824799e+15
   1.03461691e+15   1.03855474e+15   1.04323724e+15   1.04693749e+15
   1.05114763e+15   1.05599274e+15   1.06079953e+15   1.06570769e+15
   1.06946434e+15   1.07350455e+15   1.07763278e+15   1.08186759e+15]


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [52]:
rss_min = RSS.min()
print rss_min
for i in xrange(num_l1):
    #print RSS[i]
    #print np_logspace[i]
    if RSS[i]==rss_min:
        print i
        print l1_penalty_values[i]

9.66925692362e+14
0
2976351441.63


In [53]:
rss_min2 = RSS[11:15].min()
print rss_min2

1.04693748875e+15


In [54]:
print RSS[11]

1.04693748875e+15


In [55]:
print l1_penalty_values[11]

3448968612.16


In [56]:
model = graphlab.linear_regression.create(training,target='price',features=all_features,l1_penalty=l1_penalty_values[11],l2_penalty=0,validation_set=None,verbose=False)
model.get('coefficients').print_rows(num_rows=rows)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |